# Single-objective optimization with Comsol Multiphysics

The goal of this example is to show how to perform optimization with usgae of the commercial software Comsol Multiphyscics.  Note that the example is working only if there is Comsol Multiphysics locally installed and the file comsol is in the path.
The goal is to find dimensions of capacitor to reach particular value of capacity. This capacity is calculated using Comsol.


## Preparing model

The first thing is preparing the model in Comsol Multiphysics. We have to create parameters with the same name
both in the Comsol Multiphysics and consequently in Artap.

![setting_parameters](images/comsol_parameters.png)

Corresponding parameters definition in the Artap file is bellow.

```python 
self.parameters = [{'name': 'a', 'initial_value': 10, 'bounds': [0.1, 100]},  # parameters a, b must be positive
                   {'name': 'b', 'initial_value': 10, 'bounds': [0.1, 100]}]
```

---

Then, it is necessary to add a sequence to the Study:

![add_sequence](images/comsol_sequence.png)

---

The next step is to calculate the required value which is in our case: the stored energy in the capacitor. This can be done using derived values.
![derived_values](images/comsol_derived_values.png)

---
After running the simulation, the Table is created. The table must be saved in the file. Artap will use this file.

![comsol_tables](images/comsol_table.png)

At this moment, we have a model, which is prepared to use together with Artap.
We recommend clearing the solution, before you save your model.


In [ ]:
"""
Example shows how to use Artap together with localy installed Comsol Multiphysics.


"""
import os
from artap.executor import LocalComsolExecutor 
from artap.problem import Problem
from artap.algorithm_scipy import ScipyOpt


class ComsolProblem(Problem):
    """
    Describe simple one objective optimization problem.

    """

    def set(self):
        self.name = "ComsolProblem"

        # Parameters must be defined in the Comsol model
        self.parameters = [{'name': 'a', 'initial_value': 10, 'bounds': [0.1, 100]},  # parameters a, b must be positive
                           {'name': 'b', 'initial_value': 10, 'bounds': [0.1, 100]}]

        self.costs = [{'name': 'F1', 'criteria': 'minimize'}]
        self.output_files = ["energy.txt"]                                 # the names must be the same as in Comsol

        # Executor serves for calling the Comsol Multiphisics
        self.executor = LocalComsolExecutor(self,
                                            # problem_file="{}/capacitor.mph".format(os.getcwd()),  # File with the model
                                            problem_file="capacitor.mph",
                                            output_files=self.output_files)  # file with results produced by Comsol

    # Calculate the value of the objective function
    def evaluate(self, individual):
        U = 10
        C_req = 5e-12  # the capacity 5 pF is required
        energy = self.executor.eval(individual)[0]
        capacity = 2 * energy / U ** 2
        f = abs(capacity - C_req)
        print(capacity)
        return [f / C_req]  # method evaluate must return list

    # This method processes files generated by 3rd party software, depends on files format
    def parse_results(self, output_files, individual):
        output_file = output_files[0]
        path = output_file
        with open(path) as file:
            content = file.read()

        lines = content.split("\n")
        line_with_results = lines[5]  # 5th line contains results

        result = float(line_with_results.split()[2])
        return [result]


problem = ComsolProblem()  # Creating problem
algorithm = ScipyOpt(problem)  # Algorithm from Scipy was choosen
algorithm.options['algorithm'] = 'COBYLA'
algorithm.options['n_iterations'] = 5
algorithm.run()